In [1]:
from pathlib import Path
from rosbags.highlevel import AnyReader
from rosbags.typesys import Stores, get_typestore
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [2]:
imu_topic = '/inertial_sense/imu_raw'
lidar_topic = '/ouster_top/points'

def read_bag(path: Path):
    typestore = get_typestore(Stores.ROS2_HUMBLE)
    
    with AnyReader([path], default_typestore=typestore) as reader:
        connections = [x for x in reader.connections if x.topic == imu_topic or x.topic == lidar_topic]
        i = 0
        imu_data = []
        lidar_data = []
        for connection, timestamp, rawdata in reader.messages(connections=connections):
            msg = reader.deserialize(rawdata, connection.msgtype)
            if connection.topic == imu_topic:
                imu_data.append([connection.topic, timestamp, msg])
            elif connection.topic == lidar_topic:
                lidar_data.append([connection.topic, timestamp, msg])

            # print(timestamp, msg.header.frame_id, connection.msgtype, connection.topic)
            i += 1
            if i > 1000:
                break
        
        return imu_data, lidar_data

In [4]:
bag_path = Path('/home/benb/data_byu/bags/longterm_mapping/07_02_2025__0950')

imu_data, lidar_data = read_bag(bag_path)

In [5]:

imu_data[0][2]

sensor_msgs__msg__Imu(header=std_msgs__msg__Header(stamp=builtin_interfaces__msg__Time(sec=1751471675, nanosec=685783875, __msgtype__='builtin_interfaces/msg/Time'), frame_id='body', __msgtype__='std_msgs/msg/Header'), orientation=geometry_msgs__msg__Quaternion(x=0.0, y=0.0, z=0.0, w=1.0, __msgtype__='geometry_msgs/msg/Quaternion'), orientation_covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), angular_velocity=geometry_msgs__msg__Vector3(x=-5.541291830013506e-05, y=0.001095496816560626, z=-0.004641339182853699, __msgtype__='geometry_msgs/msg/Vector3'), angular_velocity_covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), linear_acceleration=geometry_msgs__msg__Vector3(x=-0.32996758818626404, y=-0.4074553847312927, z=-9.735152244567871, __msgtype__='geometry_msgs/msg/Vector3'), linear_acceleration_covariance=array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), __msgtype__='sensor_msgs/msg/Imu')

In [6]:
lidar_data[0][2]

sensor_msgs__msg__PointCloud2(header=std_msgs__msg__Header(stamp=builtin_interfaces__msg__Time(sec=1751471676, nanosec=417589238, __msgtype__='builtin_interfaces/msg/Time'), frame_id='os_lidar_top', __msgtype__='std_msgs/msg/Header'), height=64, width=1024, fields=[sensor_msgs__msg__PointField(name='x', offset=0, datatype=7, count=1, INT8=1, UINT8=2, INT16=3, UINT16=4, INT32=5, UINT32=6, FLOAT32=7, FLOAT64=8, __msgtype__='sensor_msgs/msg/PointField'), sensor_msgs__msg__PointField(name='y', offset=4, datatype=7, count=1, INT8=1, UINT8=2, INT16=3, UINT16=4, INT32=5, UINT32=6, FLOAT32=7, FLOAT64=8, __msgtype__='sensor_msgs/msg/PointField'), sensor_msgs__msg__PointField(name='z', offset=8, datatype=7, count=1, INT8=1, UINT8=2, INT16=3, UINT16=4, INT32=5, UINT32=6, FLOAT32=7, FLOAT64=8, __msgtype__='sensor_msgs/msg/PointField'), sensor_msgs__msg__PointField(name='intensity', offset=16, datatype=7, count=1, INT8=1, UINT8=2, INT16=3, UINT16=4, INT32=5, UINT32=6, FLOAT32=7, FLOAT64=8, __msgtyp

In [7]:
tmpmsg = lidar_data[0][2]
# from sensor_msgs_py import point_cloud2

In [11]:
def pointcloud2_to_array(msg):
    # msg.data is a 1D np.uint8 array
    point_step = msg.point_step
    row_step = msg.row_step
    width = msg.width
    height = msg.height
    # Reshape to (height * width, point_step) bytes
    data_reshaped = np.reshape(msg.data, (height * width, point_step))
    # Interpret x,y,z as float32 at offsets 0,4,8
    xyz = np.zeros((height*width, 3), dtype=np.float32)
    xyz[:,0] = data_reshaped[:,0:4].view(np.float32).reshape(-1)
    xyz[:,1] = data_reshaped[:,4:8].view(np.float32).reshape(-1)
    xyz[:,2] = data_reshaped[:,8:12].view(np.float32).reshape(-1)
    return xyz

xyz = pointcloud2_to_array(lidar_data[0][2])
xyz = remove_nans(xyz)
print(xyz)

Orig shape:  (65536, 3)
New shape:  (40512, 3)
[[ 18.219069  -10.307575    8.071198 ]
 [ 18.060001  -10.36437     8.028719 ]
 [ 17.842873  -10.385808    7.960319 ]
 ...
 [  4.319596    3.0432746  -2.0560954]
 [  4.3397174   3.0177765  -2.0568223]
 [  4.3558483   2.98951    -2.0557318]]


In [13]:
def pointcloud2_to_xyz(msg):
    names = []
    formats = []
    offsets = []

    for f in msg.fields:
        if f.datatype == 7:  # FLOAT32
            np_type = np.float32
        elif f.datatype == 6:  # UINT32
            np_type = np.uint32
        elif f.datatype == 4:  # UINT16
            np_type = np.uint16
        else:
            continue

        names.append(f.name)
        formats.append(np_type)
        offsets.append(f.offset)

    dt = np.dtype({
        'names': names,
        'formats': formats,
        'offsets': offsets,
        'itemsize': msg.point_step
    })

    buf = np.frombuffer(msg.data, dtype=np.uint8)
    buf = buf.reshape(msg.height * msg.width, msg.point_step)
    arr = buf.view(dt)

    xyz = np.column_stack((arr['x'], arr['y'], arr['z']))
    return xyz

In [9]:
def remove_nans(arr: np.ndarray):
    print("Orig shape: ", arr.shape)
    nan_mask = np.isnan(arr)
    rows_with_nan = nan_mask.any(axis=1)
    arr = arr[~rows_with_nan]
    print("New shape: ", arr.shape)
    return arr

xyz = pointcloud2_to_xyz(lidar_data[0][2])
# xyz = remove_nans(xyz)
print(xyz)


NameError: name 'pointcloud2_to_xyz' is not defined

In [12]:
import plotly.graph_objects as go
import numpy as np

# xyz = N x 3 numpy array
fig = go.Figure(data=[go.Scatter3d(
    x=xyz[:,0],
    y=xyz[:,1],
    z=xyz[:,2],
    mode='markers',
    marker=dict(
        size=1,
        color=xyz[:,2],      # color by height
        colorscale='Viridis',
        opacity=0.8
    )
)])
fig.show()

In [ ]:
impor